# Redes Metabólicas

##  Projeto de Alto Nível

### Componentes Principais

1. **Entrada de Dados (`ecoli.txt`)**
   - Formato: uma reação por linha
   - Exemplo: `R1: A + B => C + D`

2. **Parser de Reações**
   - Extrai dados estruturados de substratos e produtos
   - Produz uma lista de dicionários:
     ```python
     {
       "id": "R1",
       "substrates": ["A", "B"],
       "products": ["C", "D"]
     }
     ```

3. **Construção do Grafo Metabólico**
   - Usa a classe `MyGraph`
   - Cada nó representa um metabolito
   - Arestas ligam metabolitos que participam da mesma reação

4. **Módulo de Centralidade (`CentralityAnalyzer`)**
   - Calcula:
     - Centralidade de grau (Degree)
     - Centralidade de proximidade (Closeness)
     - Centralidade de intermediação (Betweenness)

5. **Módulo de Propagação de Metabolitos**
   - Dado um conjunto inicial de metabolitos:
     - Identifica reações ativas (todos os substratos disponíveis)
     - Coleta produtos gerados
     - Repete o processo iterativamente

6. **Saída**
   - Lista dos metabolitos mais centrais por métrica
   - Metabolitos finais produzíveis a partir de um conjunto inicial

---

## Projeto de Baixo Nível

### Funções Principais

####  `parse_reactions(filepath)`
- Lê `ecoli.txt` e transforma em dicionários de reação

####  `build_metabolite_graph(reactions)`
- Constrói um grafo a partir da coocorrência de metabolitos

####  `CentralityAnalyzer`
- Classe para cálculo das centralidades
- Métodos:
  - `degree_centrality()`
  - `closeness_centrality()`
  - `betweenness_centrality()`
  - `top_nodes(centrality_dict, top_n)`

####  `get_active_reactions(metabolites, reactions)`
- Retorna reações cujos substratos estão disponíveis

####  `get_produced_metabolites(active_reactions)`
- Retorna os produtos das reações ativas

####  `compute_final_metabolites(initial_metabs, reactions)`
- Aplica a propagação metabólica iterativamente


In [ ]:
from collections import deque
import heapq

class CentralityAnalyzer:
    def __init__(self, graph):
        self.graph = graph

    def degree_centrality(self):
        return {node: len(self.graph.get_successors(node)) for node in self.graph.get_nodes()}

    def closeness_centrality(self):
        centrality = {}
        for node in self.graph.get_nodes():
            total_dist = self._bfs_total_distance(node)
            centrality[node] = (len(self.graph.get_nodes()) - 1) / total_dist if total_dist > 0 else 0
        return centrality

    def _bfs_total_distance(self, start):
        visited = set()
        queue = deque([(start, 0)])
        total = 0
        while queue:
            node, dist = queue.popleft()
            if node not in visited:
                visited.add(node)
                total += dist
                for neighbor in self.graph.get_successors(node):
                    if neighbor not in visited:
                        queue.append((neighbor, dist + 1))
        return total

    def betweenness_centrality(self):
        centrality = dict.fromkeys(self.graph.get_nodes(), 0.0)
        for s in self.graph.get_nodes():
            stack = []
            pred = {w: [] for w in self.graph.get_nodes()}
            sigma = dict.fromkeys(self.graph.get_nodes(), 0)
            dist = dict.fromkeys(self.graph.get_nodes(), -1)
            sigma[s] = 1
            dist[s] = 0
            queue = deque([s])

            while queue:
                v = queue.popleft()
                stack.append(v)
                for w in self.graph.get_successors(v):
                    if dist[w] < 0:
                        dist[w] = dist[v] + 1
                        queue.append(w)
                    if dist[w] == dist[v] + 1:
                        sigma[w] += sigma[v]
                        pred[w].append(v)

            delta = dict.fromkeys(self.graph.get_nodes(), 0)
            while stack:
                w = stack.pop()
                for v in pred[w]:
                    delta[v] += (sigma[v] / sigma[w]) * (1 + delta[w])
                if w != s:
                    centrality[w] += delta[w]
        return centrality

    def top_nodes(self, centrality_dict, top_n=5):
        return heapq.nlargest(top_n, centrality_dict.items(), key=lambda x: x[1])

# REACTION PARSER
def parse_reactions(file_path):
    """Parses ecoli.txt into a list of reaction dicts"""
    reactions = []
    with open(file_path, 'r') as f:
        for line in f:
            if ':' not in line: continue
            parts = re.split(r':\s*', line.strip(), maxsplit=1)
            if len(parts) != 2: continue
            reaction_id, formula = parts
            match = re.search(r"(.*)\s*(=>|<=>)\s*(.*)", formula)
            if not match: continue
            substrates = [m.strip() for m in match.group(1).split('+')]
            products = [m.strip() for m in match.group(3).split('+')]
            reactions.append({
                'id': reaction_id,
                'substrates': substrates,
                'products': products
            })
    return reactions

def build_metabolite_graph(reactions):
    g = MyGraph()
    for r in reactions:
        metabolites = r['substrates'] + r['products']
        for i in range(len(metabolites)):
            for j in range(i + 1, len(metabolites)):
                g.add_edge(metabolites[i], metabolites[j])
    return g

# ASSESSMENT FUNCTIONS
def get_active_reactions(metabolites_set, reactions):
    return [r for r in reactions if all(sub in metabolites_set for sub in r['substrates'])]

def get_produced_metabolites(active_reactions):
    produced = set()
    for r in active_reactions:
        produced.update(r['products'])
    return produced

def compute_final_metabolites(initial_metabolites, reactions):
    known_metabolites = set(initial_metabolites)
    while True:
        active = get_active_reactions(known_metabolites, reactions)
        new = get_produced_metabolites(active)
        if new.issubset(known_metabolites):
            break
        known_metabolites.update(new)
    return known_metabolites

# MAIN EXECUTION
if __name__ == "__main__":
    file_path = "ecoli.txt"
    reactions = parse_reactions(file_path)
    
    print(" Reactions parsed:", len(reactions))

    # Centrality Analysis
    g = build_metabolite_graph(reactions)
    analyzer = CentralityAnalyzer(g)

    print("\n--- Degree Centrality ---")
    for node, val in analyzer.top_nodes(analyzer.degree_centrality()):
        print(f"{node}: {val}")

    print("\n--- Closeness Centrality ---")
    for node, val in analyzer.top_nodes(analyzer.closeness_centrality()):
        print(f"{node}: {val:.4f}")

    print("\n--- Betweenness Centrality ---")
    for node, val in analyzer.top_nodes(analyzer.betweenness_centrality()):
        print(f"{node}: {val:.4f}")

    #  Metabolite Propagation 
    initial_metabs = ["M_glc_DASH_D_c", "M_h2o_c", "M_nad_c", "M_atp_c"]
    final_metabs = compute_final_metabolites(initial_metabs, reactions)

    print("\n--- Initial Metabolites ---")
    print(initial_metabs)

    print("\n--- Final Reachable Metabolites ---")
    print(sorted(final_metabs))
